<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#1.-降维" data-toc-modified-id="1.-降维-1">1. 降维</a></span></li><li><span><a href="#2.-PCA" data-toc-modified-id="2.-PCA-2">2. PCA</a></span><ul class="toc-item"><li><span><a href="#2.1-最大化方差和最小化损失" data-toc-modified-id="2.1-最大化方差和最小化损失-2.1">2.1 最大化方差和最小化损失</a></span></li><li><span><a href="#2.2-坐标轴旋转" data-toc-modified-id="2.2-坐标轴旋转-2.2">2.2 坐标轴旋转</a></span></li></ul></li><li><span><a href="#3.-PCA-推导" data-toc-modified-id="3.-PCA-推导-3">3. PCA 推导</a></span><ul class="toc-item"><li><span><a href="#3.1-PCA-算法推导" data-toc-modified-id="3.1-PCA-算法推导-3.1">3.1 PCA 算法推导</a></span></li><li><span><a href="#3.2-维数选择" data-toc-modified-id="3.2-维数选择-3.2">3.2 维数选择</a></span></li></ul></li><li><span><a href="#4.-Sklearn-实现" data-toc-modified-id="4.-Sklearn-实现-4">4. Sklearn 实现</a></span><ul class="toc-item"><li><span><a href="#4.1-主成分可视化" data-toc-modified-id="4.1-主成分可视化-4.1">4.1 主成分可视化</a></span></li></ul></li><li><span><a href="#参考文献" data-toc-modified-id="参考文献-5">参考文献</a></span></li></ul></div>

相关文章：

[机器学习 | 目录](https://blog.csdn.net/weixin_45488228/article/details/99691709)

# 1. 降维

假设你在使用一组数据来预测房价，你的数据包含以下特征：

1. 房子面积

2. 房间数量

3. 附近学校排名

4. 社区安全

但是可以看出，1、2 在于描述房子的大小，而 3、4 在描述周边环境。因此我们可以用两个新的特征来预测房价：1. 房子大小；2. 周边环境，我们称之为`潜在变量`。现在可能我们有很多可测量的特征，但或许只有少量的潜在特征，其中包含大部分的信息。

在本文中，我们主要讨论的是如何将维度降低，降低维度的方法有两种：1.`特征选择`，2. `特征提取`。

**特征选择**指的是从已有变量中选择较少的变量，如用“房子面积”来描述“房子大小”，用“社区安全”来描述“周边环境”，如逐步回归，在 Sklearn 中用 SelectKBest 选择K个最合适的特征，或用 SelectPercentile 选取前百分比的特征。

**特征提取**则是对同一类的变量进行融合，假设现在有很多特征可以使用，但只有一分布特征在驱动这个数据模式。因此我们希望找出一个组合特征（omposite feature，又称为主成分 principle component），从而将一大堆特征缩减至几个特征。如将“房子面积”和“房间数量”融合为“房子大小”，将“附近学校排名”和“社会安全”融合为“周边环境”，这就是降维，将维数从 4 降到了 2。

`主成分分析`（Principal Component Analysis, PCA）是最常用的一种降维方法，除此之外还有增量主成分分析（IPCA）、核主成分分析（KPCA）、局部线性嵌入（LLE）、多维缩放（SDA）、等度量映射（Isomap）、t-分布随机近邻嵌入（t-SNE）和线性判别（LDA）。<sup>[1]

# 2. PCA

假设下面是房子面积和房间数量的散点图，X 轴为房子面积，Y 轴为房间数量：

<img style="float:center" src="https://x1a-alioss.oss-cn-shenzhen.aliyuncs.com/Screen Shot 2019-10-14 at 19.50.13.png" width="320" >

<center>图1 房子面积与房间数量散点图</center>

首先我们可以画出他们的主成分，如下所示：

<img style="float:center" src="https://x1a-alioss.oss-cn-shenzhen.aliyuncs.com/Screen Shot 2019-10-14 at 19.53.43.png" width="320" >

<center>图2 房子面积与房间数量的主成分 </center>

这看起来像个回归问题，但并不是这样的。在回归中，我们的目的是预测与输入值对应的输出值，在这里，我们并不是要预测任何值，而是算出数据的大致方向，使得我们的数据能够在尽量少地损失信息的同时映射在该方向上。

当我们找到了数据的主成分后，也就是向量的方向后，我们可以对数据进行一个映射，如下所示，我们的原始数据是二维的，但是，当我们把映射到主成分上之后，它就变成一维数据了。

<img style="float:center" src="https://x1a-alioss.oss-cn-shenzhen.aliyuncs.com/Screen Shot 2019-10-14 at 20.00.31.png" width="320" >

<center>图3 数据映射</center>

下面，我将介绍如何确定主成分。

## 2.1 最大化方差和最小化损失

对于我们的数据，可以用一个椭圆，这个椭圆可以用两个参数来表示：短轴的距离和长轴的距离，可以看到，在长轴上，数据更加的分散，即方差更大。我们要做的就是找到方差最大的方向。

<img style="float:center" src="https://x1a-alioss.oss-cn-shenzhen.aliyuncs.com/Screen Shot 2019-10-14 at 20.05.56.png" width="320" >

<center>图4 方差最大化</center>

为什么我们要找出具有最大方差的方向呢。换而言之，当我们在进行映射时，为什么要将所有数据点映射到方差最大的方向上呢？这是因为当我们沿着方差最大的方向进行映射时，它能够在最大程度上保留原有数据中所含信息。

当我们对数据进行映射时，就会造成信息损失，**信息的损失量就等于某个特定的点与它在这条线上的新位置之间的距离**：

<img style="float:center" src="https://x1a-alioss.oss-cn-shenzhen.aliyuncs.com/Screen Shot 2019-10-14 at 20.18.24.png" width="720" >

<center>图5 信息损失</center>

可以看到，对于图 5 的右图而言，此时数据在该方向上映射的方差最小，但信息损失也就最大。因此**当我们将方差最大化的同时，实际上是将点与其在该线上的投影之间的距离最小化，即最小化信息损失。**

## 2.2 坐标轴旋转

PCA 实际上是先对坐标轴进行旋转，通过计算数据映射在每个方向上的后的方差，选取前 K 个方差最大的特征。

# 3. PCA 推导

## 3.1 PCA 算法推导

假设输入样本为 $Z=\{Z_1,Z_2,...,Z_m\}$，我们要通过 PCA 从 $m$ 维降至 $k$ 维，首先对样本进行**中心化**，即对每个属性减去其对应的均值：

$$
\begin{aligned}
X & = \{Z_1-\bar{Z_1}, Z_2-\bar{Z_2}, ...,Z_m-\bar{Z_m} \} \\
& =  \{X_1,X_2,...,X_m\}\\
\end{aligned}\tag{1}
$$

假设经过旋转后新的坐标为 $X^T \cdot W $，其中 $W$ 为单位正交基向量矩阵（$W^T \cdot W =E$），因此要最大化投影间隔，即最大化其方差 $W^T X X^T W$（中心化后均值为 0）：

\begin{equation}
\left\{
\begin{aligned}
& \max \limits_{W} \{W^T X X^T W\} \\
& s.t. W^T W =E \\
\end{aligned}
\right.\tag{2}
\end{equation}

由拉格朗日乘数法得：

$$W^T X X^T W + \lambda [E-W^T W] \tag{3}$$

对 $W$ 求导得：

$$\frac{\partial}{\partial W}\bigg\{W^T X X^T W + \lambda [E-W^T W]\bigg\} = 2X X^T W  - \lambda E W =0 \tag{4}$$

因此有：

$$XX^Tw=\lambda W \tag{5}$$

$$(\lambda E - XX^T)W=0 \tag{6}$$

其中 $XX^T$ 为样本 $Z$ 的协方差矩阵，$\lambda$ 为 $XX^T$ 的特征值，$W$ 为 $XX^T$ 的特征向量。

将式 (5) 代入式 (2) 可得：

$$
\begin{aligned}
& max\{W^T X X^T W\} \\
= & max\{W^T\lambda W\} \\
= & max\{\lambda WW^T\} \\
= & max{\lambda E} \\
\end{aligned}\tag{7}
$$


于是，只需要对协方差矩阵 $XX^T$进行特征值分解，将求得的特征值排序：$\lambda_1 \geq,\lambda_2 \geq,...,\lambda_m$，再取前 $k$ 个特征值对应的特征向量构成投影矩阵 $W = (w_1,w_2,...,w_k)$，这就是主成分分析的解。

主成分所占整个信息的百分比可以用下式计算：

$$\frac{\sum_{i=1}^k \lambda_i}{\sum_{i=1}^m \lambda_i} \tag{8}$$

PCA 算法描述如下图所示：

<img style="float:center" src="https://x1a-alioss.oss-cn-shenzhen.aliyuncs.com/Screen Shot 2019-10-14 at 13.32.58.png" width="520" >

<center> 图6 PCA 算法</center>

> 实践中常通过对 X 进行奇异值（SVD）分解来代替协方差矩阵的特征值分解。

## 3.2 维数选择

降维后低维空间的维数 $k$ 通常是实现指定的，或通过在 $k$ 值不同的低维空间中对 KNN 分类器（或其他开销较小的学习器）进行交叉验证来选去较好的 $k$ 值。对 PCA，还可以从重构的角度设置一个重构阈值，例如 $t=95\%$，然后选择使下式成立的最小 $k$ 值：<sup>[2]

$$\frac{\sum_{i=1}^k \lambda_i}{\sum_{i=1}^m \lambda_i} \geq t \tag{9}$$

# 4. Sklearn 实现

[sklearn.decomposition.PCA](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html#sklearn.decomposition.PCA) (n_components=None, copy=True, whiten=False, svd_solver=’auto’, tol=0.0, iterated_power=’auto’, random_state=None)

**n_compoents**：主成分数量，当为 0.0 到 1.0 之间的浮点数时，表示希望保留的方差比。

***

**Methods**：

fit(self, X[, y])	Fit the model with X.

fit_transform(self, X[, y])	Fit the model with X and apply the dimensionality reduction on X.

get_covariance(self)	Compute data covariance with the generative model.

get_params(self[, deep])	Get parameters for this estimator.

get_precision(self)	Compute data precision matrix with the generative model.

**inverse_transform(self, X)**	Transform data back to its original space.

score(self, X[, y])	Return the average log-likelihood of all samples.

score_samples(self, X)	Return the log-likelihood of each sample.

set_params(self, \*\*params)	Set the parameters of this estimator.

transform(self, X)	Apply dimensionality reduction to X.

***

查看 PCA 的详细信息：

**components_**：主成分

**explained_variance_**：解释方差

**explained_variance_ratio_**：解释方差比

## 4.1 主成分可视化

``` Python
first_pc = pca.components_[0]
second_pc = pca.components_[1]

transformed_data = pca.transform(data)
for ii,jj in zip(transformed_data, data):
    plt.scatter( first_pc[0]*ii[0], first_pc[1]*ii[0], color="r")
    plt.scatter( second_pc[0]*ii[1], second_pc[1]*ii[1], color="c")
    plt.scatter( jj[0], jj[1], color="b")
   
plt.xlabel("bonus")
plt.ylabel("long-term incentive")
plt.show()
```

<img style="float:center" src="https://x1a-alioss.oss-cn-shenzhen.aliyuncs.com/IMG_22495F53D9D6-1.jpeg" width="520" >

<center>图7 安然数据集</center>

# 参考文献

[1] Aurelien Geron, 王静源, 贾玮, 边蕤, 邱俊涛. 机器学习实战：基于 Scikit-Learn 和 TensorFlow[M]. 北京: 机械工业出版社, 2018: 191.

[2] 周志华. 机器学习[M]. 北京: 清华大学出版社, 2016: 53-56.229-233 .